# Text classification using Tensorflow: Sentiment on Reviews

This notebook trains a binary classifier for sentiment analysis to classify restaurant food reviews as *negative* or *positive*, based on the text of the review where a sentiment score was previously calculated. 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade -q wandb
import wandb
from wandb.keras import WandbCallback
import os
import random
import numpy as np
import pandas as pd
import shutil
import time
import datetime
from datetime import datetime, timedelta
from timeit import default_timer as timer
import pickle
import joblib
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Nadam
import matplotlib
import matplotlib.pyplot as plt
import tqdm
from tensorflow import keras
from keras.utils import np_utils

In [ ]:
print('CUDA and NVIDIA GPU Information')
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

print('\n')
print('TensorFlow version: {}'.format(tf.__version__))
print('Eager execution is: {}'.format(tf.executing_eagerly()))
print('Keras version: {}'.format(tf.keras.__version__))
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

CUDA and NVIDIA GPU Information
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Tue Jul  5 04:43:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|            

In [ ]:
# Reproducibility in TF 
def init_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
    os.environ['TF_CUDNN_DETERMINISTIC'] ='true'
    os.environ['TF_DETERMINISTIC_OPS'] = 'true'
    tf.random.set_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                                config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    return sess

In [ ]:
init_seeds(seed=42)

In [ ]:
%cd /content/drive/MyDrive/Yelp_Reviews/Data/

/content/drive/MyDrive/Yelp_Reviews/Data


In [ ]:
# Read tokenized reviews with sentiment
df = pd.read_parquet('YelpReviews_NLP_sentimentNegPos_tokenized.parquet')
print('Number of rows and columns:', df.shape)
df.head()

Number of rows and columns: (829874, 3)


,cleanReview,sentiment,stars_reviews
index,,,
1,"[order, chicken, finger, sub, honey, mustard, ...",Negative,3.0
3,"[dedicated, loving, memory, gary, feldman, gre...",Negative,5.0
12,"[absolutely, horrible, thought, would, order, ...",Negative,1.0
15,"[found, better, chicken, finger, know, inside,...",Negative,4.0
20,"[amazing, everything, tried, disappoint, chick...",Negative,5.0


In [ ]:
# Examine how star reviews pairs with sentiment polarity
print(df[['stars_reviews', 'sentiment']].value_counts())

stars_reviews  sentiment
5.0            Positive     278011
1.0            Negative     231245
4.0            Positive     104585
2.0            Negative      90320
3.0            Negative      44097
4.0            Negative      26276
5.0            Negative      22999
3.0            Positive      22096
2.0            Positive       6690
1.0            Positive       3555
dtype: int64


In [ ]:
# Remove comma from tokenize to make one string
df = df.copy()
df['cleanReview'] = df['cleanReview'].apply(lambda x: ','.join(map(str, x)))
df.loc[:,'cleanReview'] =  df['cleanReview'].str.replace(r',', ' ', regex=True)
df.head()

,cleanReview,sentiment,stars_reviews
index,,,
1,order chicken finger sub honey mustard sauce p...,Negative,3.0
3,dedicated loving memory gary feldman greatest ...,Negative,5.0
12,absolutely horrible thought would order place ...,Negative,1.0
15,found better chicken finger know inside crisp ...,Negative,4.0
20,amazing everything tried disappoint chicken ca...,Negative,5.0


In [ ]:
# Recode to binary
df = df.copy()
df['sentiment'].mask(df['sentiment'] == 'Negative', 0, inplace=True)
df['sentiment'].mask(df['sentiment'] == 'Positive', 1, inplace=True)

print('Number of reviews per group:')
df[['sentiment']].value_counts() 

Number of reviews per group:


sentiment
0            414937
1            414937
dtype: int64

In [ ]:
# Shuffle data
df = df.drop(['stars_reviews'], axis=1)
df = shuffle(df)
df.shape

(829874, 2)

In [ ]:
# Convert data types
df = df.copy()
df[['cleanReview']] = df[['cleanReview']].astype('str')
df['sentiment'] = df['sentiment'].astype('float32')

In [ ]:
features = df['cleanReview']
labels = df['sentiment']

In [ ]:
# Load pandas df to tf
df = tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
# Define components for paritioning data
df_size = df.cardinality().numpy()
train_size = int(0.8 * df_size)
val_size = int(0.10 * df_size)
test_size = int(0.10 * df_size)
BATCH_SIZE = 32

In [ ]:
# Prepare train/test sets
df = df.shuffle(800_000, seed=42, reshuffle_each_iteration=False)
train = df.take(train_size).batch(BATCH_SIZE) 
test = df.skip(train_size)
val = test.skip(val_size).batch(BATCH_SIZE) 
test= test.take(test_size).batch(BATCH_SIZE) 

print('Train data split:', len(train))
print('Eval data split', len(val))
print('Test data split', len(test))

Train data split: 20747
Eval data split 2594
Test data split 2594


# Sentiment analysis




### Prepare the data for training

Next, the reviews are tokenized and vectorized using `tf.keras.layers.TextVectorization` layer. Tokenization refers to splitting strings into single words, or tokens. Vectorization converts the tokens into numbers as input for the neural network. Unique integer indices are created for each token by using `output_mode` = `int`. The `max_features` and `sequence_length` are defined for what is sampled.

In [ ]:
max_features = 50000
sequence_length = 300

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

The preprocessing layer is fit to the dataset by using `adapt` for the training data, which results in the model creating an index of strings to integers.

In [ ]:
# Make a review-only dataset and then call adapt
train_text = train.map(lambda x, y:x)
vectorize_layer.adapt(train_text)

Now, define a function to examine how the created layer preprocesses the data:



In [ ]:
def vectorize_text(cleanReview, sentiment):
  cleanReview = tf.expand_dims(cleanReview, -1)
  return vectorize_layer(cleanReview), sentiment

In [ ]:
# Sample a batch reviews and labels
text_batch, label_batch = next(iter(train))
first_review, first_label = text_batch[0], label_batch[0]
print('Review', first_review)
print('Label', first_label)
print('Vectorized review', vectorize_text(first_review, first_label))

Review tf.Tensor(b'waste time still business yet competition dripping springs area ordered delivery dozen time food usually okay delivery service horrible tried order three different occasion delivery told delivery driver called sick tonight placed order people told hour hour fifteen minute called hour minute later informed delivery driver leave work call replacement two hour later still food called back two people working friendly gm always mia wait another chinese place come town put business delivery fee also outrageous type service', shape=(), dtype=string)
Label tf.Tensor(0.0, shape=(), dtype=float32)
Vectorized review (<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[ 444,    7,   85,  193,  365, 2678, 2372, 2330,  123,   16,  230,
        1280,    7,    2,  288,  406,  230,    6,  122,  101,   11,  260,
         231,  936,  230,   82,  230,  789,  177,  549,  462,  616,   11,
          42,   82,   63,   63, 3115,   29,  177,   63,   29,  270, 1135,
         230,  789,  30

In [ ]:
print('12870 ---> ', vectorize_layer.get_vocabulary()[12870])
print(' 3130 ---> ', vectorize_layer.get_vocabulary()[3130])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

12870 --->  spongey
 3130 --->  unsanitary
Vocabulary size: 50000


Then, apply the `TextVectorization` layer to the train, validation, and test sets.

In [ ]:
train_ds = train.map(vectorize_text)
val_ds = val.map(vectorize_text)
test_ds = test.map(vectorize_text)

Configure the dataset for performance by using `.cache()` and `.prefetch()`.




In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Hyperparameter Tuning

In [ ]:
# Set path for results
%cd /content/drive/MyDrive/Yelp_Reviews/DL/BoW/SentimentPolarity/Models/

/content/drive/MyDrive/Yelp_Reviews/DL/BoW/SentimentPolarity/Models


In [ ]:
# Configure the sweep 
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'BinaryAccuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [3, 5, 7]
        },
        'batch_size': {
            'values': [8, 4, 2, 1]
        },
        'embedding_dim': {
            'values': [128, 64, 32]
        },        
        'dropout': {
            'values': [0.2, 0.3, 0.4, 0.5]
        },
        'weight_decay': {
            'values': [0.0005, 0.005, 0.05]
        },
        'learning_rate': {
            'values': [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop']
        },
    }
}

## 2. Initialize the Sweep

In [ ]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity='aschultz', 
                       project='tf_textclass_sentpol_r2')

Create sweep with ID: o70jyzyg
Sweep URL: https://wandb.ai/aschultz/tf_textclass_sentpol_r2/sweeps/o70jyzyg


### Define Neural Network


In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyperparameters
    config_defaults = {
        'epochs': 5,
        'batch_size': 8,
        'weight_decay': 0.0005,
        'learning_rate': 0.01,
        'optimizer': 'adam',
        'embedding_dim': 32,
        'dropout': 0.5,
        'momentum': 0.9,
        'seed': 42
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config hyperparameters and inputs
    config = wandb.config
    
    # Define the model architecture 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features + 1, config.embedding_dim))
    model.add(tf.keras.layers.Dropout(config.dropout))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dropout(config.dropout))
    model.add(tf.keras.layers.Dense(1))

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = SGD(learning_rate=config.learning_rate, decay=1e-5, 
                      momentum=config.momentum, nesterov=True)
    elif config.optimizer=='rmsprop':
      optimizer = RMSprop(learning_rate=config.learning_rate, decay=1e-5)
    elif config.optimizer=='adam':
      optimizer = Adam(learning_rate=config.learning_rate, beta_1=0.9, 
                       beta_2=0.999, clipnorm=1.0)
    elif config.optimizer=='nadam':
      optimizer = Nadam(learning_rate=config.learning_rate, beta_1=0.9, 
                        beta_2=0.999, clipnorm=1.0)

    # Define loss and metrics
    model.compile(loss=losses.BinaryCrossentropy(from_logits=True), 
                  optimizer = optimizer, 
                  metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

    # Fit model
    model.fit(train_ds, batch_size=config.batch_size,
              epochs=config.epochs,
              validation_data=(val_ds),
              callbacks=[WandbCallback(data_type='text', validation_data=(val_ds)), 
                                       EarlyStopping(patience=10, 
                                                     restore_best_weights=True)]) 

## 3. Run the sweep agent

In [ ]:
# Run sweep for HPO
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: xzcb1ebo with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 128
wandb: 	epochs: 3
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0005


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/3
20747/20747 [==============================] - 164s 8ms/step - loss: 0.6767 - binary_accuracy: 0.5933 - val_loss: 0.6637 - val_binary_accuracy: 0.6318 - _timestamp: 1656996540.0000 - _runtime: 166.0000
Epoch 2/3
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6604 - binary_accuracy: 0.6096 - val_loss: 0.6523 - val_binary_accuracy: 0.6369 - _timestamp: 1656996686.0000 - _runtime: 312.0000
Epoch 3/3
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6542 - binary_accuracy: 0.6125 - val_loss: 0.6467 - val_binary_accuracy: 0.6407 - _timestamp: 1656996832.0000 - _runtime: 458.0000


binary_accuracy,▁▇█
epoch,▁▅█
loss,█▃▁
val_binary_accuracy,▁▅█
val_loss,█▃▁
best_epoch,2
best_val_loss,0.64671
binary_accuracy,0.61253
epoch,2
loss,0.65419
val_binary_accuracy,0.64073


wandb: Agent Starting Run: 9ytlstul with config:
wandb: 	batch_size: 4
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.0005
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.05


...

wandb: Agent Starting Run: vjnbncvx with config:
wandb: 	batch_size: 1
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 64
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.05


Epoch 1/7
20747/20747 [==============================] - 115s 6ms/step - loss: 0.6908 - binary_accuracy: 0.5331 - val_loss: 0.6887 - val_binary_accuracy: 0.5534 - _timestamp: 1657059913.0000 - _runtime: 120.0000
Epoch 2/7
20747/20747 [==============================] - 115s 6ms/step - loss: 0.6873 - binary_accuracy: 0.5745 - val_loss: 0.6856 - val_binary_accuracy: 0.5849 - _timestamp: 1657060027.0000 - _runtime: 234.0000
Epoch 3/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6845 - binary_accuracy: 0.5886 - val_loss: 0.6831 - val_binary_accuracy: 0.5995 - _timestamp: 1657060141.0000 - _runtime: 348.0000
Epoch 4/7
20747/20747 [==============================] - 114s 6ms/step - loss: 0.6824 - binary_accuracy: 0.5943 - val_loss: 0.6810 - val_binary_accuracy: 0.6062 - _timestamp: 1657060255.0000 - _runtime: 462.0000
Epoch 5/7
20747/20747 [==============================] - 114s 6ms/step - loss: 0.6805 - binary_accuracy: 0.5976 - val_loss: 0.6792 - val_binary_accuracy

binary_accuracy,▁▅▇▇███
epoch,▁▂▃▅▆▇█
loss,█▆▅▄▃▂▁
val_binary_accuracy,▁▄▆▇▇██
val_loss,█▆▅▄▃▂▁
best_epoch,6
best_val_loss,0.67622
binary_accuracy,0.59965
epoch,6
loss,0.67755
val_binary_accuracy,0.61825


wandb: Agent Starting Run: 2zosvjvo with config:
wandb: 	batch_size: 1
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.005


Epoch 1/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6908 - binary_accuracy: 0.5350 - val_loss: 0.6887 - val_binary_accuracy: 0.5557 - _timestamp: 1657060732.0000 - _runtime: 119.0000
Epoch 2/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6874 - binary_accuracy: 0.5711 - val_loss: 0.6856 - val_binary_accuracy: 0.5859 - _timestamp: 1657060846.0000 - _runtime: 233.0000
Epoch 3/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6847 - binary_accuracy: 0.5823 - val_loss: 0.6831 - val_binary_accuracy: 0.5997 - _timestamp: 1657060960.0000 - _runtime: 347.0000
Epoch 4/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6826 - binary_accuracy: 0.5869 - val_loss: 0.6811 - val_binary_accuracy: 0.6065 - _timestamp: 1657061073.0000 - _runtime: 460.0000
Epoch 5/7
20747/20747 [==============================] - 114s 5ms/step - loss: 0.6809 - binary_accuracy: 0.5889 - val_loss: 0.6794 - val_binary_accuracy